In [ ]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
import rioxarray as rxr
from myfunc import timer
from myfunc import DirMan
from myfunc import load_and_flatten_data
import config

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# configuration
# resolution = "0p1"
resolution = "500"
region = [-180,180,-60,90]
data_path = f'/tera04/zhwei/xionghui/bedrock/run/{resolution}/'
post_data_path = '/tera04/zhwei/xionghui/bedrock/'
shp_path = '/tera04/zhwei/xionghui/bedrock/Shp/'
fig_path = f'/home/xuxh22/stu01/Bedrock/fig/{resolution}/'
path = '/home/xuxh22/stu01/Bedrock/'
if resolution == "0p1":
    size = 0.1
elif resolution == "500":
    size = 0.0005

dir_man = DirMan(data_path)
dir_man.enter()

os.makedirs(f'{data_path}/csv', exist_ok=True)

In [ ]:
df = pd.DataFrame()

with nc.Dataset('Sbedrock.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('Ssoil', 'Band1'),
    ('Area', 'area'),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)

df = df.dropna()
print(df['Area'].sum()/(1e12))

shp = gpd.read_file(shp_path+'World_CN/ne_10m_admin_0_countries_chn.shp')
df = df.reset_index(drop=True)
gdf_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df['lon'], df['lat']), crs='EPSG:4326')
result = gpd.sjoin(gdf_points, shp, how='left', predicate='within')

df['Sovereignt'] = result['SOVEREIGNT']
df['Sovereignt_short'] = result['ISO_A3']

df1 = df[df['Sovereignt_short'] == 'USA']
print(df1['Area'].sum()/(1e12))

df1 = df1[(df1['lat']<=49.4) & (df1['lat']>=24.5)]
df1 = df1[(df1['lon']<=-66.95) & (df1['lon']>=-124.8)]
print(df1['Area'].sum()/(1e12))
print(df1['Ssoil'].mean())
with open('csv/US_ssoil.csv','w') as f:
    df1.to_csv(f)

df1